# Thesis
## Hybrid watermarks in transform and time domain 

todo: consider Quadratic block DCT + correlation coefficient.

right now: actual watermark and transformations in python files, ipynb for testing and evaluating

Transform domain watermark: QIM in (M)DCT. This watermark is blind.

Time domain watermark: Spread Spectrum. This watermark is non-blind.

Hybrid: Currently layering. TODO: adjust watermarks to fit together better rather than just taking two unrelated watermarks and blindly layering them.

### Hybrid Info:
Assumptions: Only the originator needs the capability to decrypt and hash. (Originator = the machine that embeds the watermark)
* if others need to decrypt or check hash, asymmetric encrypted key exchange (KEK) might be the best way. this would be handled outside the watermark itself, so this project doesn't deal with that.

Actual watermark: goal right now is to hold information or an image in QIM, hash the watermarked signal, and embed the hash with LSB. for development this is a random value.

process: 
* embed dct qim watermark (blind, no info needed for decryption)
* set all LSB to 0 (temporary measure maybe - can improve this - LSB doesn't allow recovery of original signal)
* hash result
* embed hash with lsb (semi-blind - original host audio not needed but **key** and **hash seed** needed. Originator retains both.)
* to check: extract hash, zero lsbs, and check against signal. if passing, extract watermark and check info.

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import numpy as np

# Signal Processing
from scipy import signal
from scipy.fft import dct, idct, fft, ifft
from librosa import util
# from mdct import mdct, imdct 
# ^^ import error with scipy, no scipy version seems to fix it. 
# copying the functions into file "mdct_library_functions.py" for now to save time, later will either:
#   1. fix issue 
#   2. (re)write my own 
#   3. download package locally and manually edit the import 

# Audio IO
from IPython.display import Audio 
import soundfile as sf

# Plotting
import matplotlib.pyplot as plt

# Cryptography
from Crypto.Hash import MD5

# General
from time import perf_counter, perf_counter_ns
from random import randint
import sys
import struct

In [152]:
# local import
import mdct_library_functions # TODO fix
import util as watermark_utils
import window 
import dct_transform
import transform_watermark
import time_watermark
import hybrid_watermark

import evaluation

In [ ]:
# test audio load. change this
audio,sr = sf.read("hw2_audio.wav")

### Project file checks

In [46]:
# watermark_utils check
checker = np.float64(99999999999999999)
afterchange = watermark_utils.bit_change(checker, 1)
assert watermark_utils.float64_to_binary(checker)[:-2] == watermark_utils.float64_to_binary(afterchange)[:-2]
assert watermark_utils.float64_to_binary(checker)[-1] != watermark_utils.float64_to_binary(afterchange)[-1]

checker = np.float64(-99999999999999999)
afterchange = watermark_utils.bit_change(checker, 1)
assert watermark_utils.float64_to_binary(checker)[:-2] == watermark_utils.float64_to_binary(afterchange)[:-2]
assert watermark_utils.float64_to_binary(checker)[-1] != watermark_utils.float64_to_binary(afterchange)[-1]

In [5]:
# runtime check Window
audio_windowtest,sr_windowtest = sf.read("hw2_audio.wav")
audio_windowtest = util.even_audio(audio_windowtest)

window_windowtest = window.Window(1024, np.hanning)
window_windowtest.apply_window(len(audio_windowtest) // 2, audio_windowtest)

array([-0.00000000e+00, -3.43284727e-13, -3.81427474e-12, ...,
        3.99227423e-11,  1.00220069e-11,  0.00000000e+00])

In [ ]:
# runtime check MDCT
audio_mdcttest,sr_mdcttest = sf.read("hw2_audio.wav")
len_audio_mdcttest = len(audio_mdcttest)
window_mdcttest = window.Window(1024, np.hanning) # TODO sine window

print("making DCT object")
starttime_mdcttest = perf_counter()
mdct_mdcttest = dct_transform.DCT(window_mdcttest)
endtime_mdcttest = perf_counter()
print(f"DCT object timing: {endtime_mdcttest - starttime_mdcttest}\n")

print("dct")
starttime_mdcttest = perf_counter()
res_mdcttest = mdct_mdcttest.windowed_dct(audio_mdcttest)
endtime_mdcttest = perf_counter()
print(f"DCT timing: {endtime_mdcttest - starttime_mdcttest}\n")

print("idct")
starttime_mdcttest = perf_counter()
rev_mdcttest = mdct_mdcttest.windowed_idct(res_mdcttest, audio_mdcttest)
endtime_mdcttest = perf_counter()
print(f"IDCT timing: {endtime_mdcttest - starttime_mdcttest}\n")

assert len(audio_mdcttest) == len(rev_mdcttest)

# original
Audio(audio_mdcttest, rate=sr_mdcttest)

# dct and idct transformation
Audio(rev_mdcttest, rate=sr_mdcttest)

making DCT object
DCT object timing: 6.93280000003682e-05

dct
(299, 2048)
DCT object timing: 1.0600880119999996

idct
DCT object timing: 0.010300554000000517



In [167]:
# runtime check watermarkDCT
audio_wmtest,sr_wmtest = sf.read("hw2_audio.wav")

wm = "flag{this is a watermark test string let's see how it does!}"
wmbits = ''.join(format(ord(i), '08b') for i in wm)
watermark_test = transform_watermark.WatermarkDCT(np.hanning, wmbits) # random string len 45

output = watermark_test.apply_watermark(audio_wmtest)
extracted_watermark = watermark_test.extract_watermark(output, len(wmbits))

# watermark extraction check
exwmbits = "".join([str(int(x)) for x in extracted_watermark])
assert len(exwmbits) == len(wmbits)
print(exwmbits)
print(wmbits)
print(watermark_utils.visual_bitwise_check(wmbits, exwmbits)) # bitwise equals

# evaluation
robev = evaluation.RobustnessEvaluation()
print(robev.eval_all(wmbits, extracted_watermark))

# watermarked audio
Audio(output, rate=sr_wmtest)


000001001000000000000000110010000100000000000000000001100000000000000000000000000000000000000000000000000110000000000000000000000000000000000000000000000000010000000000010000100000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000001000000000000000000000000000001000000000000000000000001000000000000000000000000100001100000000000100010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
011001100110110001100001011001110111101101110100011010000110100101110011001000000110100101110011001000000110000100100000011101110110000101110100011001010111001001101101011000010111001001101011001000000111010001100101011100110111010000100000011100110111010001110010011010010110111001100111001000000110110001100101011101000010011101110011001000000111001101100101011001010010000001101000011011110111011100100000011010010111010000100000011001000110111101100101011100110010000101111101
0**001*0***0**000**0000**1*0****01***0

In [168]:
# runtime check LSB
audio_lsbtest,sr_lsbtest = sf.read("hw2_audio.wav")

wm = "flag{this is a watermark test string let's see how it does!}"
wmbits = ''.join(format(ord(i), '08b') for i in wm)
key = [(randint(0, len(audio_lsbtest)), 0) for _ in range(len(wmbits))]
lsb_lsbtest = time_watermark.LSB(key, wmbits)

watermarked_lsbtest = lsb_lsbtest.embed_watermark(audio_lsbtest)

assert not np.all(audio_lsbtest == watermarked_lsbtest) # watermarked host audio has had some change

res_lsbtest, exwm_lsbtest = lsb_lsbtest.extract_watermark(watermarked_lsbtest)
print(exwm_lsbtest)
print(wmbits)
print(watermark_utils.visual_bitwise_check(wmbits, exwm_lsbtest)) # bitwise equals visually

# evaluation
robev = evaluation.RobustnessEvaluation()
print(robev.eval_all(wmbits, exwm_lsbtest))

Audio(watermarked_lsbtest, rate=sr_wmtest)

011001100110110001100001011001110111101101110100011010000110100101110011001000000110100101110011001000000110000100100000011101110110000101110101011001010111001001101101011000010111001001101011001000000111010001100101011100110111010000100000011100110111010001110010011010010110111001100111001000000110110001100101011101000010011101110011001000000111001101100101011001010010000001101000011011110111011100100000011010010111010000100000011001000110111101100101011100110010000101111101
011001100110110001100001011001110111101101110100011010000110100101110011001000000110100101110011001000000110000100100000011101110110000101110100011001010111001001101101011000010111001001101011001000000111010001100101011100110111010000100000011100110111010001110010011010010110111001100111001000000110110001100101011101000010011101110011001000000111001101100101011001010010000001101000011011110111011100100000011010010111010000100000011001000110111101100101011100110010000101111101
01100110011011000110000101100111011110

In [169]:
# Hybrid checks

# prep
audio_hybrid,sr_hybrid = sf.read("hw2_audio.wav")

wm = "flag{this is a watermark test string let's see how it does!}"
wmbits = ''.join(format(ord(i), '08b') for i in wm)
watermark_test = transform_watermark.WatermarkDCT(np.hanning, wmbits) # random string len 45

# run watermark
hybrid = hybrid_watermark.Hybrid(watermark_test)
real_hash, lsb_lsbtest, embedded = hybrid.embed_watermark(audio_hybrid)
extracted_hash, signal_output, extracted_watermark = hybrid.extract_watermark(embedded, lsb_lsbtest, len(wmbits))

# checks
print(extracted_hash)
print(real_hash)
# check lsb
assert extracted_hash == real_hash
audio_hash_checker = hybrid.md5_audio(signal_output)
assert audio_hash_checker == extracted_hash
# print(watermark_utils.visual_bitwise_check(audio_hash_checker, exwm_lsbtest))

# check dct/qim
exwmbits = "".join([str(int(x)) for x in extracted_watermark])
assert len(exwmbits) == len(wmbits)
print(exwmbits)
print(wmbits)
print(watermark_utils.visual_bitwise_check(wmbits, exwmbits))

# evaluation
robev = evaluation.RobustnessEvaluation()
print(robev.eval_all(real_hash, extracted_hash))

# evaluation
robev = evaluation.RobustnessEvaluation()
print(robev.eval_all(wmbits, exwmbits))

11111101000011110001111001001001000001011000001101111100001100110100010010000011000000011100100101111011011100101110001010111100
11111101000011110001111001001001000001011000001101111100001100110100010010000011000000011100100101111011011100101110001010111100
000001001000000000000000110010000100000000000000000001100000000000000000000000000000000000000000000000000110000000000000000000000000000000000000000000000000010000000000010000100000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000001000000000000000000000000000001000000000000000000000001000000000000000000000000100001100000000000100010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
011001100110110001100001011001110111101101110100011010000110100101110011001000000110100101110011001000000110000100100000011101110110000101110100011001010111001001101101011000010111001001101011001000000111010001100101011100110111010000100000011100110111010001110

In [179]:
# Evaluation checks
audio_eval,sr_eval = sf.read("hw2_audio.wav")
wm = "flag{this is a watermark test string let's see how it does!}"
wmbits = ''.join(format(ord(i), '08b') for i in wm)

# Hybrid
watermark_test = transform_watermark.WatermarkDCT(np.hanning, wmbits)
hybrid = hybrid_watermark.Hybrid(watermark_test)
real_hash, lsb_lsbtest, embedded = hybrid.embed_watermark(audio_hybrid)

print("hybrid eval check")
rob_eval = evaluation.RobustnessEvaluation()
all_results = rob_eval.run_suite(embedded, sr_eval, (hybrid, lsb_lsbtest), hash=real_hash, wmbits=wmbits)

# Transform
watermark_test = transform_watermark.WatermarkDCT(np.hanning, wmbits)
embedded = watermark_test.apply_watermark(audio_hybrid)

print("transform eval check")
rob_eval = evaluation.RobustnessEvaluation()
all_results = rob_eval.run_suite(embedded, sr_eval, watermark_test, wmbits=wmbits)

# Time
key = [(randint(0, len(audio_lsbtest)), 0) for _ in range(len(wmbits))]
watermark_test = time_watermark.LSB(key, wmbits) # TODO: is evaluation unfair if i use a hash for hybrid but wmbits for time?
embedded = watermark_test.embed_watermark(audio_hybrid)

print("time eval check")
rob_eval = evaluation.RobustnessEvaluation()
all_results = rob_eval.run_suite(embedded, sr_eval, watermark_test, hash=wmbits)

hybrid eval check
((1.0, 1.0, np.float64(1.0), 0.0), (0.044444444444444446, 0.45454545454545453, np.float64(-0.006238581174591127), 0.47291666666666665))
((1.0, 1.0, np.float64(1.0), 0.0), (0.044444444444444446, 0.45454545454545453, np.float64(-0.006238581174591127), 0.47291666666666665))
((1.0, 1.0, np.float64(1.0), 0.0), (0.044444444444444446, 0.45454545454545453, np.float64(-0.006238581174591127), 0.47291666666666665))
((1.0, 1.0, np.float64(1.0), 0.0), (0.044444444444444446, 0.45454545454545453, np.float64(-0.006238581174591127), 0.47291666666666665))
((1.0, 1.0, np.float64(1.0), 0.0), (0.044444444444444446, 0.45454545454545453, np.float64(-0.006238581174591127), 0.47291666666666665))
((1.0, 1.0, np.float64(1.0), 0.0), (0.044444444444444446, 0.45454545454545453, np.float64(-0.006238581174591127), 0.47291666666666665))
transform eval check
([], (0.044444444444444446, 0.45454545454545453, np.float64(-0.006238581174591127), 0.47291666666666665))
([], (0.044444444444444446, 0.454545454

In [ ]:
 # Scrapped early code, keeping for posterity 
    


   
    # 
    # mdct: calculates and outputs mdct transformation
    # imdct: calculates and outputs imdct transformation
    # def mdct(self, x): # todo ensure x is even length
    #     """
    #     mdct Computes MDCT of a signal x of length 2*N. Appends one 0 to end if odd length.

    #     Parameters
    #     ----------
    #     x : array_like
    #         data to transform

    #     Returns
    #     -------
    #     array_like 
    #         MDCT transformed signal
    #     """
    #     x_e = even_audio(x)
    #     N = len(x_e) // 2

    #     res = np.zeros(N)
    #     for ix in range(N):
    #         res[ix] = np.sum([x_e[i]*np.cos((np.pi / N) * (i + .5 + N/2) * (ix + .5)) for i in range(2*N)])
    #     return res

    # def imdct(self, y):
    #     """
    #     imdct Computes inverse MDCT of a signal y

    #     Parameters
    #     ----------
    #     y : array_like
    #         data to transform

    #     Returns
    #     -------
    #     array_like 
    #         inverse MDCT transformed signal
    #     """
    #     N = len(y)
    #     res = np.zeros(2*N)
    #     for iy in range(2*N):
    #         res[iy] = np.sum([y[i]*np.cos((np.pi/N) * (iy + .5 + N/2) * (i + .5)) for i in range(N)])*2/N
    #     print(len(res))
    #     return res
    
    # def windowed_mdct(self, x):
    #     """
        
    #     """
    #     x_e = even_audio(x)
    #     N = self.window.win_len
    #     hop_size = N # for 50% overlap
    #     num_windows = max(1, (len(x_e) - 2*N) // hop_size + 1)

    #     res = np.zeros((num_windows, N))

    #     for i in range(num_windows):
    #         if i % 20 == 0:
    #             print("DEBUG: " + str(i) + " out of " + str(num_windows))

    #         curr = i*hop_size
    #         if curr + 2*N <= len(x_e): # have not reached end
    #             subarray = x_e[curr:curr+2*N]
    #         else: # have reached end
    #             subarray = np.zeros(2*N)
    #             subarray[:len(x_e)-curr] = x_e[curr:]

    #         windowed_subarray = self.window.apply_window(N, subarray)

    #         res[i, :] = self.mdct(windowed_subarray)
    #     return res
    
    



    # # matrix product
    # def windowed_imdct(self, y, original_length):
    #     """
        
    #     """
    #     num_windows, N = y.shape
    #     hop_size = N # for 50% overlap

    #     res = np.zeros(original_length)
    #     overlap = np.zeros(original_length)

    #     for i in range(num_windows):
    #         if i % 20 == 0:
    #             print(f"DEBUG: {i} out of {num_windows}")

    #         subarray = self.imdct(y[i])
    #         windowed_subarray = self.window.apply_window(N, subarray)

    #         curr = i * hop_size
    #         if curr + 2*N <= original_length: # have not reached end 
    #             res[curr:curr+2*N] += windowed_subarray
    #             overlap[curr:curr+2*N] += np.ones(2*N)
    #         else:    # have reached end, ignore extra
    #             res[curr:] += windowed_subarray[:original_length-curr]
    #             overlap[curr:] += np.ones(original_length - curr)
            
    #     overlap[overlap == 0] = 1
    #     res /= overlap

    #     return res



# # TODO GET IRD OF THIS
# output = watermark_test.apply_watermark(audio_hybrid)

# # ready for hashing
# zerod_output = watermark_utils.zero_lsb(output) # TODO: might interfere with QIM, figure out a dif way to make this blind-ish
# ready_output = zerod_output.tobytes()

# # hash 
# md5 = MD5.new()
# md5.update(ready_output)
# audio_hash = watermark_utils.bytestring_to_bitstring(md5.digest())

# # LSB
# key = [(randint(0, sys.maxsize), 0) for _ in range(len(audio_hash))]
# lsb_lsbtest = time_watermark.LSB(key, audio_hash)

# watermarked_lsbtest = lsb_lsbtest.embed_watermark(zerod_output)
# TODO GET IRD OF THIS

# extracted_hash, signal_output, extracted_watermark = hybrid.extract_watermark(watermarked_lsbtest)

# assert extracted_hash == audio_hash

# extract
# res_lsbtest, exwm_lsbtest = lsb_lsbtest.extract_watermark(watermarked_lsbtest)
# zerod_output = watermark_utils.zero_lsb(res_lsbtest)
# ready_output = zerod_output.tobytes()

# md5checker = MD5.new()
# md5checker.update(ready_output)
# audio_hash_checker = watermark_utils.bytestring_to_bitstring(md5checker.digest())

# extracted_watermark = watermark_test.extract_watermark(zerod_output, len(wmbits))
